In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

## The heat equation
Solve $$\partial_t u = \frac{a}{2}\Delta u$$ 

for a function $u = u(\mathbf{x}, t)$, where $\mathbf{x}=(x_1, \ldots, x_d)$, $\Delta \equiv \sum_{k=1}^d \partial^2_{x_k x_k}$, and $a>0$.

### Standard boundary/initial condition problems

Boundary Value Problem: 
* given $u|_{\mathbf{x}\in \Omega, t=0}$ and $u|_{\mathbf{x}\in \partial\Omega, t>0}$
* solve for $u|_{\mathbf{x}\in \Omega, t>0}.$

Initial Value Problem in $\mathbb{R}^d\times[0,+\infty)$:
* given a decaying $u(\mathbf{x},t)|_{\mathbf{x}\in \mathbb{R}^d, t=0}$ or $u(\mathbf{x},t)|_{\mathbf{x}\in \mathbb{R}^d, t=0}$
* solve for $u|_{\mathbf{x}\in \mathbb{R}^d, t>0}.$

Initial Value Problem in $[0,1]^d\times[0,+\infty)$ with periodic b.c.:
* given a $u(\mathbf{x},t)|_{\mathbf{x}\in [0,1]^d, t=0}$ 
* solve for $u|_{\mathbf{x}\in [0,1]^d, t>0}$ assuming periodic b.c. (i.e. on torus $\mathbb T=\mathbb S^d$, where $\mathbb S=[0,1]/\{0\equiv 1\}$).

### Theoretical solution of IVP in $\mathbb{R}^d$
$$u(\mathbf{x},t)=u(\cdot,t=0)*\phi_{at}=\int_{\mathbb R^d} u(\mathbf y,t=0)\phi_{at}(\mathbf{x}-\mathbf{y})d\mathbf{y},$$
where
$$\phi_{at}(\mathbf{x}) = \frac{1}{(2\pi at)^{d/2}}e^{-\frac{|\mathbf{x}|^2}{2at}}$$

**Exercise:** Notable properties of the solution of heat equation:
* $u\in C^\infty(\mathbb R^d\times (0,+\infty))$ (the solution is infinitely smooth at $t>0$ even if the initial value was not smooth);
* $\|u\|_\infty\le \|u(\cdot,t=0)\|_\infty$ (solution preserves the $L_\infty$-norm); 
* $\int_{\mathbb R^d} u(\mathbf x, t) d\mathbf x = \int_{\mathbb R^d} u(\mathbf x, t=0) d\mathbf x$ for all $t$ ("heat conservation"); 
* $\min_{\mathbf x}u(\mathbf x, t)$ is a monotone non-decreasing function of $t$; 
* $\max_{\mathbf x}u(\mathbf x, t)$ is a monotone non-increasing function of $t$ 

### Solving heat eq. on torus numerically

Define some discretization and initial conditions.

In [ ]:
u0 = 2-np.sqrt(np.sum(plt.imread('sk.png')**2, axis=2))
N = u0.shape[0]
dx = 1./N
#u0 = np.exp(-70*(4*(x0-0.3)**2+(x1-0.1)**2))+0.5*np.exp(-20*((x0-0.6)**2+(x1-0.4)**2)) + 

%matplotlib notebook
plt.imshow(u0, cmap='inferno')
plt.colorbar()
plt.show()

### First attempt at solution: forward Euler iterations
Let $t_n=n\Delta t$ and $\mathbf x_{\mathbf m}=(m_1\Delta x, \ldots, m_d\Delta x)$, with some $\Delta t, \Delta x$. 

Forward Euler iterations:
$$\tilde u(\cdot, t_{n+1}) = \tilde u(\cdot, t_{n})+  \frac{a}{2}\tilde{\Delta}  u(\cdot, t_{n})\Delta t,$$
where $$\tilde{\Delta}u(\mathbf x_{\mathbf m})\equiv\frac{1}{(\Delta x)^2} \Big(\sum_{k=1}^d \big(u(\mathbf x_{\mathbf m+\mathbf e_k})+u(\mathbf x_{\mathbf m-\mathbf e_k})\big)-2d u(\mathbf x_{\mathbf m})\Big)$$

Plot min/max/mean of solution as functions of $t$:

Validate expected properties of the solution:

Animation:

In [ ]:
def animateSolution(uHistory, dt, skip):
    tHistory = dt*skip*np.arange(len(uHistory))
    fig = plt.figure()
    im = plt.imshow(uHistory[0], vmin=0, vmax=1, cmap='inferno' #'Greys_r'
                    , animated=True)
    plt.colorbar()
    title = plt.title('')

    def animate(i): 
        im.set_array(uHistory[i]) 
        title.set_text("t = %2.5f" %(tHistory[i]))

    ani = animation.FuncAnimation(fig, animate, len(uHistory))
    plt.show()
    return ani

%matplotlib auto
ani = animateSolution(uHistory, dt, skip)

### Why does solution blow up unless $dt$ is very small?

Spectral analysis: consider Fourier expansion
$$u(\mathbf x_{\mathbf m}) = \frac{1}{N^d} \sum_{\mathbf p} \check u_{\mathbf p} e^{\frac{2\pi i \mathbf p\cdot \mathbf m}{N}},$$
where $\mathbf p = (p_1, \ldots, p_d)\in \mathbb Z_N^d.$

**Exercise:** The function $e^{\frac{2\pi i \mathbf p\cdot \mathbf m}{N}}$ is an eigenvector of the Laplacian $\tilde \Delta$ with eigenvalue $c_{\mathbf p} = \frac{2}{(\Delta x)^2}\sum_{k=1}^d \big(\cos \frac{2\pi p_k}{N}-1\big) \le 0.$

The Euler evolution of $u$ decouples into independent evolutions of coefficients $\check u_{\mathbf p}$: 
$$\check u_{\mathbf p}(t_{n+1}) = \check u_{\mathbf p}(t_{n})+\frac{a\Delta t}{2}c_{\mathbf p}\check u_{\mathbf p}(t_{n})$$
that are supposed to solve the linear first order ODE
$$\frac{d}{dt}\check u_{\mathbf p}(t) = \frac{a}{2}c_{\mathbf p}\check u_{\mathbf p}(t).$$

The true solution of this equation is $$\check u_{\mathbf p}(t) = \check u_{\mathbf p}(t=0)e^{(ac_{\mathbf p}t)/2}.$$

Approximate solution using forward Euler is 
$$\tilde{\check u}_{\mathbf p}(t_n) = \check u_{\mathbf p}(t=0)\Big(1+\frac{a \Delta t c_{\mathbf p}}{2}\Big)^n.$$

Therefore, for the approximate Euler solution to be close to the true solution we need 

$$\Big(1+\frac{a c_{\mathbf p}\Delta t}{2}\Big)^n\approx e^{(ac_{\mathbf p}n\Delta t)/2}.$$

Since $c_{\mathbf p}\le 0$, the r.h.s. is nonnegative and non-increasing, but the l.h.s. will not even be bounded unless the stability condition $$\Big|1+\frac{a c_{\mathbf p}\Delta t}{2}\Big|\le 1$$ holds.

Since $\min_{\mathbf p}c_{\mathbf p}=-\frac{4d}{(\Delta x)^2},$ to avoid blow-up of the Euler solution of heat equation we need $$\Delta t \lesssim \frac{(\Delta x)^2}{da}.$$

**Exercise:** Derive the stability condition for the wave equation $$\partial^2_{tt} u = a\Delta u.$$

**Exercise:** Implement solution for BVP.

### Implicit (backward) Euler method

Backward Euler iterations:
$$\tilde u(\cdot, t_{n+1}) = \tilde u(\cdot, t_{n})+  \frac{a}{2}\tilde{\Delta}  u(\cdot, t_{n+1})\Delta t.$$

* Advantage: always stable (**exercise**).
* Drawback: requires us to invert the matrix $1-\frac{a\Delta t}{2}\tilde{\Delta}$: 
$$\tilde u(\cdot, t_{n+1}) = \Big(1-\frac{a\Delta t}{2}\tilde{\Delta}\Big)^{-1}\tilde u(\cdot, t_{n}).$$

Approximate inversion can be performed efficiently by an iterative method using sparse structure of $\tilde{\Delta}$.

In general, to solve $Ax=b$ iteratively, we may write $A=A_1+A_2$, where
* $A_1$ is easy to invert (e.g., diagonal or triangular);
* $A_2$ is small in some sense w.r.t. $A_1$.

Then we write our iterations as 

$$A_1 x_{n+1}+A_2 x_n = b$$

or, equivalently,

$$x_{n+1}=A_1^{-1}(b-A_2x_n)=A_1^{-1}b-A_1^{-1}A_2x_n.$$

If $A=1-\frac{a\Delta t}{2}\tilde{\Delta}$, then $\operatorname{spec}A\subset\big[1,1+\frac{2ad\Delta t}{(\Delta x)^2}\big]$ and it is reasonable to choose $$A_1 = 1+\frac{ad\Delta t}{(\Delta x)^2}$$ so that $$A_2=-\frac{a\Delta t}{2}\Big(\tilde{\Delta}+\frac{2d}{(\Delta x)^2}\Big),$$
$$A_2 u(\mathbf x_{\mathbf m})=-\frac{a\Delta t}{2(\Delta x)^2} \sum_{k=1}^d \big(u(\mathbf x_{\mathbf m+\mathbf e_k})+u(\mathbf x_{\mathbf m-\mathbf e_k})\big).$$

**Exercise:** Show that with this choice of $A_1$ the operator $A_1^{-1}A_2$ is contractive and 

$$\|A_1^{-1}A_2\|\le 1-\frac{1}{1+\frac{ad\Delta t}{(\Delta x)^2}}$$

in the $l_2$ operator norm. Estimate convergence of the iterates to the true solution $\Big(1-\frac{a\Delta t}{2}\tilde{\Delta}\Big)^{-1}\tilde u(\cdot, t_{n})$.


Test of the backward Euler:

In [ ]:
%matplotlib auto
animateSolution(uHistory, dt, skip)

In [ ]:
%matplotlib auto
animateSolution(uHistory, dt, skip)

**Exercise:** Implement a stable solver for
* Wave equation
* Burgers equation

**Exercise:** How fast is the distributed simulation compared to the non-distributed one? Experiment with different simulation settings and explain the results.

**Exercise:** Implement distributed simulation with 4 processes. 